In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib  
from selenium.webdriver.support.select import Select
import time     
from bs4 import BeautifulSoup
import pandas as pd
import PyPDF2 as pdf
import lxml
sleep_time=2

In [2]:
def pdf2text():
    pdfFileObj = open('Lawfile.pdf', 'rb')
    pdfReader = pdf.PdfFileReader(pdfFileObj)
    pagenum=pdfReader.numPages
    text=''
    for i in range(pagenum):
        pageObj = pdfReader.getPage(i)
        text+=' '+pageObj.extractText() 
    pdfFileObj.close()
    return text

In [68]:
def Home(name='Administration'):
    # uploading chromedrive file 
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    browser = webdriver.Chrome('./chromedriver',chrome_options=chrome_options)

    # openning URL in chromedriver 
    URL='https://caselaw.findlaw.com/summary/search'
    browser.get(URL)
    browser.maximize_window()

    # Preparing Home page scrapeable 
    time.sleep(sleep_time)
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight / 4)')
    time.sleep(sleep_time)
    select=Select(browser.find_element_by_name('topic'))
    time.sleep(sleep_time)
    soup=BeautifulSoup(browser.page_source,'lxml')
    legal_name={i:[optiontag['value'],optiontag.text] for i,optiontag in enumerate(soup.find('select',attrs={'id':'topic'}).find_all('option')[1:])}
    print(legal_name)
    n=int(input('Select a LawName key in integer : '))
    select.select_by_value(legal_name[n][0])
    time.sleep(sleep_time)
    searchbtn=browser.find_element_by_class_name('primary')
    searchbtn.click()
    return browser

In [4]:
def parse_data(raw_html):
    label=raw_html.find('i').text
    desc=raw_html.find('div',attrs={'class':'desc_subttl'}).text
    link=raw_html.find('a')['href']
    date=raw_html.find_all('td')[-2].text
    return [desc,link,date,label]

def scrape_text(browser,url):
    try:
        browser.window_handles[1]
    except Exception as e:
        browser.execute_script("window.open('');")
    browser.switch_to.window(browser.window_handles[1])
    browser.get(url)
    time.sleep(sleep_time)
    tinybtn=browser.find_element_by_link_text('READ').click()
    time.sleep(sleep_time)
    try:
        herebtn=browser.find_element_by_link_text('here')
        herebtn.click()
        browser.switch_to.window(browser.window_handles[2])
        url=browser.current_url
        file = urllib.request.urlopen(url)
        pdffile = open('Lawfile.pdf', 'wb')
        pdffile.write(file.read())
        text=pdf2text()
    except Exception as e:
        text=browser.find_element_by_class_name('caselawcontent').text
#     browser.close()
    browser.switch_to.window(browser.window_handles[1])
    return text

def ScrapeLinksAndSaveData(browser):
    global counter
    soup=BeautifulSoup(browser.page_source,'lxml')
    fileLinks=soup.find_all('tr')
    for i in range(1,len(fileLinks)):
        trlst=parse_data(fileLinks[i])
        text=scrape_text(browser,trlst[1])
        trlst.append(text)
        df.loc[counter]=trlst
        counter+=1

In [75]:
df=pd.DataFrame(columns=['Description','url','Date','Label','Text'])
counter=0
browser=Home()
totalPages=int(browser.find_element_by_class_name('pagination').text.split('\n')[-2])
pages=0

{0: ['cs_1', 'Administrative Law'], 1: ['cs_2', 'Admiralty'], 2: ['cs_3', 'Agriculture'], 3: ['cs_4', 'Antitrust & Trade Regulation'], 4: ['cs_79', 'Asset Forfeiture'], 5: ['cs_5', "Attorney's Fees"], 6: ['cs_6', 'Banking Law'], 7: ['cs_7', 'Bankruptcy Law'], 8: ['cs_8', 'Civil Procedure'], 9: ['cs_9', 'Civil Rights'], 10: ['cs_10', 'Class Actions'], 11: ['cs_11', 'Commercial Law'], 12: ['cs_12', 'Communications Law'], 13: ['cs_13', 'Constitutional Law'], 14: ['cs_14', 'Construction'], 15: ['cs_15', 'Consumer Protection Law'], 16: ['cs_16', 'Contracts'], 17: ['cs_17', 'Corp. Governance'], 18: ['cs_18', 'Corporation & Enterprise Law'], 19: ['cs_19', 'Criminal Law & Procedure'], 20: ['cs_20', 'Habeas Corpus'], 21: ['cs_21', 'Sentencing'], 22: ['cs_22', 'Cyberspace Law'], 23: ['cs_23', 'Debt Collection'], 24: ['cs_24', 'Dispute Resolution & Arbitration'], 25: ['cs_25', 'Drugs & Biotech'], 26: ['cs_31', 'ERISA'], 27: ['cs_26', 'Education Law'], 28: ['cs_27', 'Elder Law'], 29: ['cs_28', 'El

In [76]:

while pages!=totalPages:
    try:
        ScrapeLinksAndSaveData(browser)
        pass
    except Exception as e:
        print(e)
    browser.switch_to.window(browser.window_handles[0])
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)/2')
    browser.find_element_by_class_name('pagination-next').find_element_by_tag_name('a').click()
    try:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight)/4')
    except Exception as e:
        pass
    pages+=1

Message: no such element: Unable to locate element: {"method":"css selector","selector":".caselawcontent"}
  (Session info: chrome=85.0.4183.83)

Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=85.0.4183.83)



WebDriverException: Message: chrome not reachable
  (Session info: chrome=85.0.4183.83)


In [78]:
df

,Description,url,Date,Label,Text
0,California Court of Appeal,https://caselaw.findlaw.com/summary/opinion/ca...,08/02/2019,"White Collar Crime, Insurance Law, Workers' Co...","People v. Pierce, 074602\nRead People v. Pierc..."
1,United States Ninth Circuit,https://caselaw.findlaw.com/summary/opinion/us...,06/07/2019,"White Collar Crime, Criminal Law & Procedure\n...",FOR PUBLICATION\n UNITED STATES COURT OF APPE...
2,California Court of Appeal,https://caselaw.findlaw.com/summary/opinion/ca...,05/22/2019,"White Collar Crime, Property Law & Real Estate...","People v. Astorga-Lider, 073992\nRead People v..."
3,United States Ninth Circuit,https://caselaw.findlaw.com/summary/opinion/us...,05/02/2019,"Health Law, White Collar Crime, Criminal Law &...",FOR PUBLICATION\n UNITED STATES COURT OF APPE...
4,United States Fifth Circuit,https://caselaw.findlaw.com/summary/opinion/us...,04/30/2019,"White Collar Crime, Criminal Law & Procedure\n...",IN THE UNITED STATES COURT OF APPEALS\n FOR TH...
5,United States Seventh Circuit,https://caselaw.findlaw.com/summary/opinion/us...,02/12/2019,"White Collar Crime, Criminal Law & Procedure\n...",In the\n United States Court of Appeals\n For ...
6,United States Seventh Circuit,https://caselaw.findlaw.com/summary/opinion/us...,02/07/2019,"Health Law, White Collar Crime, Criminal Law &...",In theUnited States Court of AppealsFor the Se...
7,United States Seventh Circuit,https://caselaw.findlaw.com/summary/opinion/us...,12/14/2018,"Criminal Law & Procedure, Sentencing, White Co...",In the\n United States Court of Appeals\n Fo...
8,United States Seventh Circuit,https://caselaw.findlaw.com/summary/opinion/us...,12/10/2018,"Criminal Law & Procedure, Sentencing, White Co...",In the\n United States Court of Appeals\n Fo...
9,Supreme Court of California,https://caselaw.findlaw.com/summary/opinion/ca...,12/10/2018,"White Collar Crime, Criminal Law & Procedure\n...","People v. Franco, 233973\nRead People v. Franc..."


In [ ]:
# chrome_options = webdriver.ChromeOptions()
# prefs = {"profile.default_content_setting_values.notifications" : 2}
# chrome_options.add_experimental_option("prefs",prefs)
# browser = webdriver.Chrome('./chromedriver',chrome_options=chrome_options)

# URL='https://caselaw.findlaw.com/summary/search'

# browser.get(URL)
# browser.maximize_window()
# time.sleep(sleep_time)
# browser.execute_script('window.scrollTo(0, document.body.scrollHeight / 4)')
# time.sleep(sleep_time)
# select=Select(browser.find_element_by_name('topic'))
# time.sleep(sleep_time)
# select.select_by_value('cs_1')
# time.sleep(sleep_time)
# searchbtn=browser.find_element_by_class_name('primary')
# searchbtn.click()
# time.sleep(sleep_time)
# ttlbtn=browser.find_element_by_link_text('Lockett v. Bonson').click()
# time.sleep(sleep_time)
# tinybtn=browser.find_element_by_link_text('READ').click()
# time.sleep(sleep_time)
# herebtn=browser.find_element_by_link_text('here')
# herebtn.click()
# browser.switch_to.window(browser.window_handles[1])
# url=browser.current_url
# file = urllib.request.urlopen(url)
# pdffile = open('Lawfile.pdf', 'wb')
# pdffile.write(file.read())
# file.close()
# browser.quit()